In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

In [12]:
# Carregar os dados
customers_df = pd.read_csv('../data/olist_customers_dataset.csv')
geolocation_df = pd.read_csv('../data/olist_geolocation_dataset.csv')
orders_items_df = pd.read_csv('../data/olist_order_items_dataset.csv')
payments_df = pd.read_csv('../data/olist_order_payments_dataset.csv')
reviews_df = pd.read_csv('../data/olist_order_reviews_dataset.csv')
orders_df = pd.read_csv('../data/olist_orders_dataset.csv')
products_df = pd.read_csv('../data/olist_products_dataset.csv')
sellers_df = pd.read_csv('../data/olist_sellers_dataset.csv')

In [13]:
# 1. Calcule a média de lat/lng por prefixo
geo_mean = geolocation_df.groupby('geolocation_zip_code_prefix')[['geolocation_lat', 'geolocation_lng']].mean().reset_index()

# 2. Adicione ao customers_df
customers_geo = pd.merge(customers_df, geo_mean, left_on='customer_zip_code_prefix', right_on='geolocation_zip_code_prefix', how='left')
customers_geo = customers_geo.rename(columns={'geolocation_lat': 'customer_lat', 'geolocation_lng': 'customer_lng'})

# 3. Adicione ao sellers_df
sellers_geo = pd.merge(sellers_df, geo_mean, left_on='seller_zip_code_prefix', right_on='geolocation_zip_code_prefix', how='left')
sellers_geo = sellers_geo.rename(columns={'geolocation_lat': 'seller_lat', 'geolocation_lng': 'seller_lng'})

In [16]:
geolocation_df = geolocation_df[['geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng']]

# Fazer o merge entre os datasets
merged_df = pd.merge(orders_items_df, products_df, on='product_id', how='inner')
merged_df = pd.merge(merged_df, orders_df, on='order_id', how='inner')
merged_df = pd.merge(merged_df, reviews_df, on='order_id', how='inner')
merged_df = pd.merge(merged_df, customers_df, on='customer_id', how='inner')
merged_df = pd.merge(merged_df, sellers_df, on='seller_id', how='inner')
merged_df = pd.merge(merged_df, payments_df, on='order_id', how='inner')
# Selecionar apenas as colunas de coordenadas e id para evitar duplicidade
merged_df = pd.merge(merged_df, customers_geo[['customer_id', 'customer_lat', 'customer_lng']], on='customer_id', how='inner')
merged_df = pd.merge(merged_df, sellers_geo[['seller_id', 'seller_lat', 'seller_lng']], on='seller_id', how='inner')


# criar uma nova coluna: tempo de entrega
merged_df['order_purchase_timestamp'] = pd.to_datetime(merged_df['order_purchase_timestamp'])
merged_df['order_delivered_customer_date'] = pd.to_datetime(merged_df['order_delivered_customer_date'])
merged_df['delivery_time'] = (merged_df['order_delivered_customer_date'] - merged_df['order_purchase_timestamp']).dt.days

# criando feature volume do produto
merged_df['product_volume'] = merged_df['product_length_cm'] * merged_df['product_height_cm'] * merged_df['product_width_cm']


# criando feature dia da semana
merged_df['order_purchase_dayofweek'] = merged_df['order_purchase_timestamp'].dt.dayofweek

In [17]:
merged_df.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_category_name,product_name_lenght,product_description_lenght,...,payment_type,payment_installments,payment_value,customer_lat,customer_lng,seller_lat,seller_lng,delivery_time,product_volume,order_purchase_dayofweek
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,cool_stuff,58.0,598.0,...,credit_card,2,72.19,-21.762775,-41.309633,-22.496953,-44.127492,7.0,3528.0,2
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,pet_shop,56.0,239.0,...,credit_card,3,259.83,-20.220527,-50.903424,-23.565096,-46.518565,16.0,60000.0,2
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,moveis_decoracao,59.0,695.0,...,credit_card,5,216.87,-19.870305,-44.593326,-22.262584,-46.171124,7.0,14157.0,6
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,perfumaria,42.0,480.0,...,credit_card,2,25.78,-23.089925,-46.611654,-20.553624,-47.387359,6.0,2400.0,2
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,ferramentas_jardim,59.0,409.0,...,credit_card,3,218.04,-23.243402,-46.827614,-22.929384,-53.135873,25.0,42000.0,5


In [18]:
# listando todas as colunas e o tipo de dado
print(merged_df.dtypes)

order_id                                 object
order_item_id                             int64
product_id                               object
seller_id                                object
shipping_limit_date                      object
price                                   float64
freight_value                           float64
product_category_name                    object
product_name_lenght                     float64
product_description_lenght              float64
product_photos_qty                      float64
product_weight_g                        float64
product_length_cm                       float64
product_height_cm                       float64
product_width_cm                        float64
customer_id                              object
order_status                             object
order_purchase_timestamp         datetime64[ns]
order_approved_at                        object
order_delivered_carrier_date             object
order_delivered_customer_date    datetim

In [42]:
# One-Hot Encoding para variáveis categóricas
df_model_encoded = pd.get_dummies(merged_df, columns=['customer_state', 'seller_state'], drop_first=True)

In [45]:
df_model_encoded.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_category_name,product_name_lenght,product_description_lenght,...,seller_state_PE,seller_state_PI,seller_state_PR,seller_state_RJ,seller_state_RN,seller_state_RO,seller_state_RS,seller_state_SC,seller_state_SE,seller_state_SP
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,cool_stuff,58.0,598.0,...,False,False,False,False,False,False,False,False,False,True
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,pet_shop,56.0,239.0,...,False,False,False,False,False,False,False,False,False,True
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,moveis_decoracao,59.0,695.0,...,False,False,False,False,False,False,False,False,False,False
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,perfumaria,42.0,480.0,...,False,False,False,False,False,False,False,False,False,True
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,ferramentas_jardim,59.0,409.0,...,False,False,True,False,False,False,False,False,False,False


--------------------------------------------------

In [32]:
# Pré-processamento de dados
# Remover valores ausentes ou imputar, conforme necessário
df_model_encoded = merged_df.dropna(subset=['delivery_time'])  # Exemplo de remoção de dados faltantes

# Codificação de variáveis categóricas (One-Hot Encoding)
# df_model_encoded = pd.get_dummies(df_model_cleaned, columns=['customer_state', 'seller_state'], drop_first=True)

# Codificação para cateogria de produto
df_model_encoded['product_category_name'] = df_model_encoded['product_category_name'].astype('category')
df_model_encoded['product_category_name'] = df_model_encoded['product_category_name'].cat.codes

# feature estado do vendedor e cliente
df_model_encoded['customer_state'] = df_model_encoded['customer_state'].astype('category')
df_model_encoded['customer_state'] = df_model_encoded['customer_state'].cat.codes
df_model_encoded['seller_state'] = df_model_encoded['seller_state'].astype('category')
df_model_encoded['seller_state'] = df_model_encoded['seller_state'].cat.codes


# feature de distância geográfica entre cliente e vendedor
df_model_encoded['customer_lat'] = df_model_encoded['customer_lat'].astype('float')
df_model_encoded['customer_lng'] = df_model_encoded['customer_lng'].astype('float')
df_model_encoded['seller_lat'] = df_model_encoded['seller_lat'].astype('float')
df_model_encoded['seller_lng'] = df_model_encoded['seller_lng'].astype('float')

# usando distancia haversine para calcular a distancia entre dois pontos geográficos
def haversine(lat1, lon1, lat2, lon2):
    # Conversão de graus para radianos
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    # Fórmula de Haversine
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    r = 6371  # Raio da Terra em quilômetros
    return c * r

df_model_encoded['distance'] = haversine(
    df_model_encoded['customer_lat'], 
    df_model_encoded['customer_lng'], 
    df_model_encoded['seller_lat'], 
    df_model_encoded['seller_lng']
)



# feature cidade
df_model_encoded['customer_city'] = df_model_encoded['customer_city'].astype('category')
df_model_encoded['customer_city'] = df_model_encoded['customer_city'].cat.codes
df_model_encoded['seller_city'] = df_model_encoded['seller_city'].astype('category')
df_model_encoded['seller_city'] = df_model_encoded['seller_city'].cat.codes

# feature mes do ano
df_model_encoded['order_purchase_month'] = df_model_encoded['order_purchase_timestamp'].dt.month.astype('category')
df_model_encoded['order_purchase_month'] = df_model_encoded['order_purchase_month'].cat.codes

# feature diferença em dias entre shipping_limit_date e order_purchase_timestamp
df_model_encoded['shipping_limit_date'] = pd.to_datetime(df_model_encoded['shipping_limit_date'])
df_model_encoded['order_purchase_timestamp'] = pd.to_datetime(df_model_encoded['order_purchase_timestamp'])
df_model_encoded['shipping_limit_days'] = (df_model_encoded['shipping_limit_date'] - df_model_encoded['order_purchase_timestamp']).dt.days


# encode pagamento tipo
df_model_encoded['payment_type'] = df_model_encoded['payment_type'].astype('category')
df_model_encoded['payment_type'] = df_model_encoded['payment_type'].cat.codes


Q1 = df_model_encoded['delivery_time'].quantile(0.25)
Q3 = df_model_encoded['delivery_time'].quantile(0.75)
IQR = Q3 - Q1

# Mantém apenas valores dentro de [Q1 - 1.5*IQR, Q3 + 1.5*IQR]
df_model_filtered = df_model_encoded[
    (df_model_encoded['delivery_time'] >= Q1 - 1.5 * IQR) & 
    (df_model_encoded['delivery_time'] <= Q3 + 1.5 * IQR)
]

# Seleção das features e variável alvo
features = ['price', 'freight_value', 'product_photos_qty', 'product_volume', 'review_score', 'order_purchase_dayofweek', 'seller_state', 'customer_state', 'seller_city', 'customer_city', 'shipping_limit_days', 'payment_type', 'order_purchase_month', 'distance']
X = df_model_encoded[features]
y = df_model_encoded['delivery_time']

C:\Users\Edilton\AppData\Local\Temp\ipykernel_12496\3855062900.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model_encoded['product_category_name'] = df_model_encoded['product_category_name'].astype('category')
C:\Users\Edilton\AppData\Local\Temp\ipykernel_12496\3855062900.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model_encoded['product_category_name'] = df_model_encoded['product_category_name'].cat.codes
C:\Users\Edilton\AppData\Local\Temp\ipykernel_12496\3855062900.py:13: SettingWit

In [33]:
df_model_encoded.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_category_name,product_name_lenght,product_description_lenght,...,customer_lat,customer_lng,seller_lat,seller_lng,delivery_time,product_volume,order_purchase_dayofweek,distance,order_purchase_month,shipping_limit_days
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,26,58.0,598.0,...,-21.762775,-41.309633,-22.496953,-44.127492,7.0,3528.0,2,301.504681,8,6
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,63,56.0,239.0,...,-20.220527,-50.903424,-23.565096,-46.518565,16.0,60000.0,2,585.563937,3,7
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,54,59.0,695.0,...,-19.870305,-44.593326,-22.262584,-46.171124,7.0,14157.0,6,312.343511,0,4
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,62,42.0,480.0,...,-23.089925,-46.611654,-20.553624,-47.387359,6.0,2400.0,2,293.168420,7,7
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,40,59.0,409.0,...,-23.243402,-46.827614,-22.929384,-53.135873,25.0,42000.0,5,646.163463,1,9


In [34]:
# Divisão entre treino e teste (80% treino, 20% teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalonamento dos dados (StandardScaler)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [35]:
# Construção e treinamento do modelo Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train_scaled, y_train)

RandomForestRegressor(random_state=42)

In [36]:
# Previsões
y_pred = rf_model.predict(X_test_scaled)

# Avaliação do modelo
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae:.2f} dias")
print(f"RMSE: {rmse:.2f} dias")
print(f"R²: {r2:.2%}")

MAE: 3.79 dias
RMSE: 6.70 dias
R²: 48.40%
